In [1]:
import sys
sys.path.append("src")

In [2]:
import hydra
import torch

from torch.optim import Adam
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from hydra import compose, initialize

In [3]:
with initialize(version_base="1.3", config_path="configs"):
    cfg = compose(config_name="train")

In [4]:
print(cfg)

{'task_name': 'train', 'callbacks': {'checkpoint': {'_target_': 'lightning.pytorch.callbacks.ModelCheckpoint', 'monitor': 'val_loss', 'dirpath': 'logs/${task_name}/runs/${now:%Y-%m-%d}/${now:%H-%M-%S}/checkpoint/', 'filename': 'ner-{epoch:02d}-{val_loss:.2f}', 'save_top_k': 1, 'mode': 'min'}, 'early_stop': {'_target_': 'lightning.pytorch.callbacks.early_stopping.EarlyStopping', 'monitor': 'val_acc', 'mode': 'max', 'verbose': True, 'patience': 5}}, 'data': {'_target_': 'ner.data.ConllDataModule', 'data_path': 'data/conll', 'max_len': 100}, 'logger': {'mlflow_logger': {'_target_': 'lightning.pytorch.loggers.mlflow.MLFlowLogger', 'tracking_uri': 'logs/mlflow/', 'experiment_name': 'ner', 'log_model': True}, 'tensorboard_logger': {'_target_': 'lightning.pytorch.loggers.tensorboard.TensorBoardLogger', 'save_dir': 'logs/tensorboard/', 'name': 'ner'}}, 'loss': {'_target_': 'torch.nn.CrossEntropyLoss', 'ignore_index': 1}, 'metrics': {'_target_': 'torchmetrics.Accuracy', 'task': 'multiclass', 'n

In [5]:
from ner.model import GruModule

In [6]:
module = GruModule(
    model = cfg.model,
    loss_fn = cfg.loss,
    metrics = cfg.metrics,
    optim = cfg.optimizer
)

In [7]:
module.model

GruNER(
  (emb): Embedding(26776, 256)
  (gru): GRU(256, 128, num_layers=2, batch_first=True, bidirectional=True)
  (out_layer): Linear(in_features=128, out_features=9, bias=True)
)

In [8]:
instantiate(cfg.module, _recursive_=False)

GruModule(
  (model): GruNER(
    (emb): Embedding(26776, 256)
    (gru): GRU(256, 128, num_layers=2, batch_first=True, bidirectional=True)
    (out_layer): Linear(in_features=128, out_features=9, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
  (metrics): MulticlassAccuracy()
)